In [3]:
import argparse
import tensorflow as tf

from tuner import HyperparameterTuner

from tensorflow.contrib import tpu
from tensorflow.contrib.cluster_resolver import TPUClusterResolver

In [4]:
hidden_layers = 2
hidden_units = 500
num_perms = 2
trials = 10
epochs = 10
task_home = 'gs://continual_learning/permMNIST_EWC/'
checkpoint_path = task_home + 'logs/checkpoints/'
summaries_path = task_home + 'logs/summaries/'
data_path = task_home + 'MNIST_data/'
tpu_name = 'gectpu'

In [5]:
tpu_cluster = TPUClusterResolver(tpu=[tpu_name]).get_master()
sess = tf.Session(tpu_cluster)
sess.run(tpu.initialize_system())

b'\n\x03\x02\x02\x02\x10\x01\x18\x08"\x18\x00\x00\x00\x00\x00\x01\x00\x01\x00\x00\x01\x01\x01\x00\x00\x01\x00\x01\x01\x01\x00\x01\x01\x01'

In [6]:
tuner = HyperparameterTuner(sess=sess, hidden_layers=hidden_layers, hidden_units=hidden_units,
                                        num_perms=num_perms, trials=trials, epochs=epochs,
                                        checkpoint_path=checkpoint_path, summaries_path=summaries_path, 
                                        data_path=data_path)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting gs://continual_learning/permMNIST_EWC/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting gs://continual_learning/permMNIST_EWC/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting gs://continual_learning/permMNIST_EWC/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting gs://continual_learning/permMNIST_EWC/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.

In [7]:
from queue import PriorityQueue

In [6]:
# task 0
t = 0
queue = PriorityQueue()
for learning_rate in tuner.trial_learning_rates:
    tuner.train_on_task(t, learning_rate, queue)
tuner.best_parameters.append(queue.get())

training layers=2,hidden=500,lr=0.00094,multiplier=1067.63,mbsize=250,epochs=10,perm=0 with weights initialized at None
saving model layers=2,hidden=500,lr=0.00094,multiplier=1067.63,mbsize=250,epochs=10,perm=0 at time step 1119
finished training layers=2,hidden=500,lr=0.00094,multiplier=1067.63,mbsize=250,epochs=10,perm=0
INFO:tensorflow:Restoring parameters from gs://continual_learning/permMNIST_EWC/logs/checkpoints/layers=2,hidden=500,lr=0.00094,multiplier=1067.63,mbsize=250,epochs=10,perm=0.ckpt-1119
training layers=2,hidden=500,lr=0.00038,multiplier=2599.17,mbsize=250,epochs=10,perm=0 with weights initialized at None
saving model layers=2,hidden=500,lr=0.00038,multiplier=2599.17,mbsize=250,epochs=10,perm=0 at time step 1119
finished training layers=2,hidden=500,lr=0.00038,multiplier=2599.17,mbsize=250,epochs=10,perm=0
INFO:tensorflow:Restoring parameters from gs://continual_learning/permMNIST_EWC/logs/checkpoints/layers=2,hidden=500,lr=0.00038,multiplier=2599.17,mbsize=250,epochs=

In [8]:
# accuracy on validation sets
t = 1
tuner.best_parameters = []
tuner.best_parameters.append((0.00077, 'layers=2,hidden=500,lr=0.00077,multiplier=1291.96,mbsize=250,epochs=10,perm=0'))
print("task0", tuner.classifier.test(sess=tuner.sess, model_name=tuner.best_parameters[t - 1][1], batch_xs=tuner.task_list[0].validation.images, batch_ys=tuner.task_list[0].validation.labels))
print("task1", tuner.classifier.test(sess=tuner.sess, model_name=tuner.best_parameters[t - 1][1], batch_xs=tuner.task_list[1].validation.images, batch_ys=tuner.task_list[1].validation.labels))

INFO:tensorflow:Restoring parameters from gs://continual_learning/permMNIST_EWC/logs/checkpoints/layers=2,hidden=500,lr=0.00077,multiplier=1291.96,mbsize=250,epochs=10,perm=0.ckpt-1119
task0 0.9910086
INFO:tensorflow:Restoring parameters from gs://continual_learning/permMNIST_EWC/logs/checkpoints/layers=2,hidden=500,lr=0.00077,multiplier=1291.96,mbsize=250,epochs=10,perm=0.ckpt-1119
task1 0.0


In [9]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

In [10]:
# task1
t = 1

N_learning_rates = 10
learning_rates = 10.0 ** np.arange(-9, 2)
# lr = 0.00094
# 1e19 works good - [62, 72] for 15000
#1e22 - [69, 68] for 15000
# fisher_multiplier = 1e24
fisher_multipliers = 10.0 ** np.arange(18, 28)
dataset_train = tuner.task_list[t].train
dataset_lagged = tuner.task_list[t - 1].train if t > 0 else None
model_init_name = tuner.best_parameters[t - 1][1] if t > 0 else None
MINI_BATCH_SIZE = 256
LOG_FREQUENCY = 100
dataset_train.initialize_iterator(MINI_BATCH_SIZE)
if (dataset_lagged is not None):
    dataset_lagged.initialize_iterator(MINI_BATCH_SIZE)

eval_frequency = 100
num_updates = 3000

result = {}

In [11]:
import time

In [20]:
num_updates = 5000
fisher_multipliers = np.logspace(4, 6, 20)
learning_rates = [5e-6]
best_avg = 0.0
best_params = -1
for fisher_multiplier in fisher_multipliers:
    for lr in learning_rates:
        print("fisher_multiplier: %e, lr: %e" % (fisher_multiplier, lr))
        start_time = time.time()
        model_name = tuner.file_name(lr, t)
        tuner.classifier.prepare_for_training(sess=tuner.sess, 
                                            model_name=model_name, 
                                            model_init_name=model_init_name, 
                                            fisher_multiplier=fisher_multiplier, 
                                            learning_rate=lr)
        val_acc = [[], []]
        val_loss = [[], []]
        loss = []
        loss_with_penalty = []
        cur_best_avg = 0.0
        cur_best_avg_num_updates = -1
        i = 0
        count_not_improving = 0
        while (True):
            cur_loss, cur_loss_with_penalty = tuner.classifier.minibatch_sgd(tuner.sess, i, dataset_train, MINI_BATCH_SIZE, LOG_FREQUENCY)
            loss.append(cur_loss)
            loss_with_penalty.append(cur_loss_with_penalty)
            if (i % eval_frequency == 0):
                cur_iter_avg = 0.0
                for j in range(num_perms):
                    val_data = tuner.task_list[j].validation
                    feed_dict = tuner.classifier.create_feed_dict(val_data.images, val_data.labels, keep_input=1.0, keep_hidden=1.0)
                    accuracy = sess.run([tuner.classifier.loss, tuner.classifier.accuracy], feed_dict=feed_dict)
                    val_loss[j].append(accuracy[0])
                    val_acc[j].append(accuracy[1])
                    cur_iter_avg += accuracy[1]
                cur_iter_avg /= num_perms
                
                if (val_acc[0][-1] < val_acc[1][-1]):
                    if (cur_best_avg >= cur_iter_avg):
                        count_not_improving += 1
                    else:
                        count_not_improving = 0
                        
                if (cur_iter_avg > cur_best_avg):
                    cur_best_avg = cur_iter_avg
                    cur_best_avg_num_updates = i
                
                if (count_not_improving >= 5):
                    break
            i += 1
        result[(fisher_multiplier, lr)] = {}
        result[(fisher_multiplier, lr)]["val_acc"] = val_acc
        result[(fisher_multiplier, lr)]["val_loss"] = val_loss
        result[(fisher_multiplier, lr)]["loss"] = loss
        result[(fisher_multiplier, lr)]["loss_with_penalty"] = loss_with_penalty
        result[(fisher_multiplier, lr)]["best_avg"] = (cur_best_avg, cur_best_avg_num_updates)
        if (best_avg < cur_best_avg):
            best_avg = cur_best_avg
            best_params = (fisher_multiplier, lr)
        print("time taken: %f" % (time.time() - start_time))
        print("loss with penalty: %f, loss: %f, val0 accuracy: %f, val1 accuracy: %f"
              % (loss_with_penalty[-1], loss[-1], 
                 val_acc[0][cur_best_avg_num_updates // eval_frequency], val_acc[1][cur_best_avg_num_updates // eval_frequency]))

fisher_multiplier: 1.000000e+04, lr: 5.000000e-06
INFO:tensorflow:Restoring parameters from gs://continual_learning/permMNIST_EWC/logs/checkpoints/layers=2,hidden=500,lr=0.00077,multiplier=1291.96,mbsize=250,epochs=10,perm=0.ckpt-1119
time taken: 49.730911
loss with penalty: 0.724411, loss: 0.459475, val0 accuracy: 0.922205, val1 accuracy: 0.821867
fisher_multiplier: 1.274275e+04, lr: 5.000000e-06
INFO:tensorflow:Restoring parameters from gs://continual_learning/permMNIST_EWC/logs/checkpoints/layers=2,hidden=500,lr=0.00077,multiplier=1291.96,mbsize=250,epochs=10,perm=0.ckpt-1119
time taken: 53.957276
loss with penalty: 0.746294, loss: 0.446002, val0 accuracy: 0.926114, val1 accuracy: 0.819820
fisher_multiplier: 1.623777e+04, lr: 5.000000e-06
INFO:tensorflow:Restoring parameters from gs://continual_learning/permMNIST_EWC/logs/checkpoints/layers=2,hidden=500,lr=0.00077,multiplier=1291.96,mbsize=250,epochs=10,perm=0.ckpt-1119
time taken: 53.473317
loss with penalty: 0.750146, loss: 0.4136

MemoryError: 

In [12]:
best_params = (10 ** (18 / 19 + 4), 5e-6)

In [23]:
tuner_before_training = HyperparameterTuner(sess=sess, hidden_layers=hidden_layers, hidden_units=hidden_units,
                                            num_perms=num_perms, trials=trials, epochs=epochs,
                                            checkpoint_path=checkpoint_path, summaries_path=summaries_path, 
                                            data_path=data_path)

Extracting gs://continual_learning/permMNIST_EWC/MNIST_data/train-images-idx3-ubyte.gz
Extracting gs://continual_learning/permMNIST_EWC/MNIST_data/train-labels-idx1-ubyte.gz
Extracting gs://continual_learning/permMNIST_EWC/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting gs://continual_learning/permMNIST_EWC/MNIST_data/t10k-labels-idx1-ubyte.gz


In [28]:
# confusion matrix before training
# train on best hyperparameters
fisher_multiplier, lr = best_params

print("fisher_multiplier: %e, lr: %e" % (fisher_multiplier, lr))
start_time = time.time()
model_name = tuner.file_name(lr, t)
tuner.classifier.prepare_for_training(sess=tuner.sess, 
                                    model_name=model_name, 
                                    model_init_name=model_init_name, 
                                    fisher_multiplier=fisher_multiplier, 
                                    learning_rate=lr)

num_labels = 10
pred = np.array([])
actual = np.array([])
for j in range(num_perms):
    val_data = tuner.task_list[j].validation
    feed_dict = tuner.classifier.create_feed_dict(val_data.images, val_data.labels, keep_input=1.0, keep_hidden=1.0)
    cur_scores, cur_y = tuner.classifier.get_confusion_matrix(sess, feed_dict)
    cur_pred = np.argmax(cur_scores, 1)
    cur_actual = np.argmax(cur_y, 1)
    print(cur_actual)
    actual = np.concatenate([actual, cur_actual])
    pred = np.concatenate([pred, cur_pred])
confusion_matrix = np.zeros((num_labels,num_labels), dtype=np.int64)

for i in range(actual.shape[0]):
    confusion_matrix[int(actual[i]), int(pred[i])] += 1

print("%3d" % (0, ), end=' ')
for j in range(confusion_matrix.shape[1]):
    print("%3d" % (j, ), end=' ')
print("")
for i in range(confusion_matrix.shape[0]):
    print("%3d" % (i, ), end=' ')
    for j in range(confusion_matrix.shape[1]):
        print("%3d" % (confusion_matrix[i][j], ), end= ' ')
    print("")


fisher_multiplier: 8.858668e+04, lr: 5.000000e-06
INFO:tensorflow:Restoring parameters from gs://continual_learning/permMNIST_EWC/logs/checkpoints/layers=2,hidden=500,lr=0.00077,multiplier=1291.96,mbsize=250,epochs=10,perm=0.ckpt-1119
[0 4 1 ... 2 1 2]
[5 9 5 ... 7 8 7]
  0   0   1   2   3   4   5   6   7   8   9 
  0 477   0   0   1   1   0   0   0   0   0 
  1   0 558   2   1   2   0   0   0   0   0 
  2   4   3 480   1   0   0   0   0   0   0 
  3   0   0   4 488   1   0   0   0   0   0 
  4   0   2   1   0 532   0   0   0   0   0 
  5  46  20   4 292  72   0   0   0   0   0 
  6  53  35 171  13 229   0   0   0   0   0 
  7  42  27 100 213 168   0   0   0   0   0 
  8  13  81  68 266  34   0   0   0   0   0 
  9   4   5   2  14 470   0   0   0   0   0 


In [13]:
# train on best hyperparameters
fisher_multiplier, lr = best_params

print("fisher_multiplier: %e, lr: %e" % (fisher_multiplier, lr))
start_time = time.time()
model_name = tuner.file_name(lr, t)
tuner.classifier.prepare_for_training(sess=tuner.sess, 
                                    model_name=model_name, 
                                    model_init_name=model_init_name, 
                                    fisher_multiplier=fisher_multiplier, 
                                    learning_rate=lr)
val_acc = [[], []]
val_loss = [[], []]
loss = []
loss_with_penalty = []
cur_best_avg = 0.0
cur_best_avg_num_updates = -1
i = 0
count_not_improving = 0
while (True):
    cur_loss, cur_loss_with_penalty = tuner.classifier.minibatch_sgd(tuner.sess, i, dataset_train, MINI_BATCH_SIZE, LOG_FREQUENCY)
    loss.append(cur_loss)
    loss_with_penalty.append(cur_loss_with_penalty)
    if (i % eval_frequency == 0):
        cur_iter_avg = 0.0
        for j in range(num_perms):
            val_data = tuner.task_list[j].validation
            feed_dict = tuner.classifier.create_feed_dict(val_data.images, val_data.labels, keep_input=1.0, keep_hidden=1.0)
            accuracy = sess.run([tuner.classifier.loss, tuner.classifier.accuracy], feed_dict=feed_dict)
            val_loss[j].append(accuracy[0])
            val_acc[j].append(accuracy[1])
            cur_iter_avg += accuracy[1]
        cur_iter_avg /= num_perms

        if (val_acc[0][-1] < val_acc[1][-1]):
            if (cur_best_avg >= cur_iter_avg):
                count_not_improving += 1
            else:
                count_not_improving = 0

        if (cur_iter_avg > cur_best_avg):
            cur_best_avg = cur_iter_avg
            cur_best_avg_num_updates = i

        if (count_not_improving >= 5):
            break
    i += 1
result[(fisher_multiplier, lr)] = {}
result[(fisher_multiplier, lr)]["val_acc"] = val_acc
result[(fisher_multiplier, lr)]["val_loss"] = val_loss
result[(fisher_multiplier, lr)]["loss"] = loss
result[(fisher_multiplier, lr)]["loss_with_penalty"] = loss_with_penalty
result[(fisher_multiplier, lr)]["best_avg"] = (cur_best_avg, cur_best_avg_num_updates)
print("time taken: %f" % (time.time() - start_time))
print("loss with penalty: %f, loss: %f, val0 accuracy: %f, val1 accuracy: %f"
      % (loss_with_penalty[-1], loss[-1], 
         val_acc[0][cur_best_avg_num_updates // eval_frequency], val_acc[1][cur_best_avg_num_updates // eval_frequency]))

fisher_multiplier: 8.858668e+04, lr: 5.000000e-06
INFO:tensorflow:Restoring parameters from gs://continual_learning/permMNIST_EWC/logs/checkpoints/layers=2,hidden=500,lr=0.00077,multiplier=1291.96,mbsize=250,epochs=10,perm=0.ckpt-1119
time taken: 71.609308
loss with penalty: 0.950625, loss: 0.438365, val0 accuracy: 0.910477, val1 accuracy: 0.871826


In [ ]:
print("best_avg: %e, best_params: %s" % (best_avg, str(best_params)))

for k, v in result.items():
    cur_res = v
    x = np.arange(0, len(cur_res['loss']), eval_frequency)
    cur_best_avg = cur_res['best_avg']
    print("fisher_multiplier: %e, lr: %e" % (k[0], k[1]))
    print("cur_best_avg: %e, num_updates: %e" % (cur_best_avg[0], cur_best_avg[1]))
    print("val0_acc: %e, val1_acc: %e" %
        (cur_res['val_acc'][0][cur_best_avg[1] // eval_frequency], cur_res['val_acc'][1][cur_best_avg[1] // eval_frequency]))
    plt.plot(cur_res['loss_with_penalty'], color='g')
    plt.plot(cur_res['loss'], color='m')
    plt.plot(x, cur_res['val_loss'][1], color='b')
    plt.show()
    plt.plot(cur_res['val_acc'][0], color='b')
    plt.plot(cur_res['val_acc'][1], color='g')
    plt.show()

In [21]:
# confusion matrix 
# finding which digits are being confused by classifier
num_labels = 10
pred = np.array([])
actual = np.array([])
for j in range(num_perms):
    val_data = tuner.task_list[j].validation
    feed_dict = tuner.classifier.create_feed_dict(val_data.images, val_data.labels, keep_input=1.0, keep_hidden=1.0)
    cur_scores, cur_y = tuner.classifier.get_confusion_matrix(sess, feed_dict)
    cur_pred = np.argmax(cur_scores, 1)
    cur_actual = np.argmax(cur_y, 1)
    print(cur_actual)
    actual = np.concatenate([actual, cur_actual])
    pred = np.concatenate([pred, cur_pred])
confusion_matrix = np.zeros((num_labels,num_labels), dtype=np.int64)

for i in range(actual.shape[0]):
    confusion_matrix[int(actual[i]), int(pred[i])] += 1

print("%3d" % (0, ), end=' ')
for j in range(confusion_matrix.shape[1]):
    print("%3d" % (j, ), end=' ')
print("")
for i in range(confusion_matrix.shape[0]):
    print("%3d" % (i, ), end=' ')
    for j in range(confusion_matrix.shape[1]):
        print("%3d" % (confusion_matrix[i][j], ), end= ' ')
    print("")

[0 4 1 ... 2 1 2]
[5 9 5 ... 7 8 7]
  0   0   1   2   3   4   5   6   7   8   9 
  0 451   0   0   0   0  16   9   1   2   0 
  1   0 552   1   0   1   1   2   1   5   0 
  2   1   4 443   1   0   4  13   2  20   0 
  3   0   1   3 395   0  53   0   8  29   4 
  4   0   1   1   0 381   0  18   0   0 134 
  5   1   3   1  12   0 378  11   2  18   8 
  6   2   3   2   0   1   8 480   0   5   0 
  7   4   5   7   0   3   5   7 495   2  22 
  8   1   6   4   0   0  35   7   2 394  13 
  9   2   4   1   3   1   6   5  15   8 450 


In [29]:
sess.run(tpu.shutdown_system())